In [139]:
root_dir = '/home2/glee/dissertation/1_tech_gen_impact/class2class/Tech_Gen/'
master_dir = '/home2/glee/dissertation/1_tech_gen_impact/master/Tech_Gen/'
import sys
sys.path.append(root_dir)

import copy
import gc
import os
import argparse
import math
import time
import pickle
import re
import multiprocess as mp
import warnings
warnings.filterwarnings(action='ignore', category=UserWarning)
warnings.filterwarnings(action='ignore', category=DeprecationWarning)
sys.path.append("/share/tml_package")
from tml import utils
from scipy import io
from tqdm import tqdm
from collections import OrderedDict

import torch
from torch.nn import functional as F
from torch.nn import DataParallel as DP
from torch.utils.data import TensorDataset, DataLoader, Subset, Dataset
from accelerate import Accelerator
import pytorch_model_summary

import optuna
from optuna.samplers import RandomSampler, TPESampler
from optuna.integration import SkoptSampler

import numpy as np
import pandas as pd
import scipy.stats
import sklearn
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import matthews_corrcoef, precision_recall_fscore_support, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight

from data import TechDataset, CVSampler
from models import Transformer, Predictor
from train_utils import EarlyStopping, perf_eval, objective_cv, build_model, train_model, validate_model_mp
from utils import token2class, DotDict, to_device

from cleantext.sklearn import CleanTransformer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

# PART 1: Configuration

In [214]:
temp = pd.read_csv(os.path.join(data_dir, "collection_" + "".join([config_keywords, config_ipcs, config_period]) + ".csv"), index_col=0)

/home2/glee/.conda/envs/DL/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (0,1) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [215]:
temp

,number,year,month,assignee,main cpc,sub cpc,main ipc,sub ipc,<1976,1976,...,2016,2017,2018,2019,2020,2021,2022,claims,claims_org,forward_refs
8925188,8925188,2015,1,Fujitsu Limited,H05K3/305,Y10T29/53183;H05K2203/1115;Y10T29/53191;Y10T29...,B23P19/00,H05K3/30,0,0,...,1,1,1,4,30,1,1,1. A mounting apparatus comprising: a stage de...,1. A mounting apparatus comprising: a stage de...,10932401;9210836;9761556
8925190,8925190,2015,1,Panasonic Intellectual Property Management Co....,H05K13/083;H05K13/0812;H05K13/0452,Y10T29/53174;Y10T29/49904;H01L2224/81;Y10T29/5...,H05K3/30,H05K13/04,0,0,...,1,1,1,1,30,1,1,1. An electronic component mounting device whi...,1. An electronic component mounting device whi...,10271437
8925260,8925260,2015,1,P4P Holdings LLC,H02S20/32;H02S20/10;F24S25/50;E04C3/30;H02S20/20,Y02E10/47;Y02E10/50,E04H12/20,H01L31/042,0,0,...,1,1,1,1,30,1,1,1. A method of supporting a solar panel array ...,1. A method of supporting a solar panel array ...,10337586;9184694;9954478
8925262,8925262,2015,1,Building Materials Investment Corporation,E04D1/30;H02S40/34;E04D13/174;H02S20/23;E04D13...,Y02E10/50;Y02B10/10;E04D2001/305,E04D5/00,F24F7/02;E04D3/00;H01L31/048;E04D13/10,0,0,...,1,1,1,1,2,1,1,1. A ridge vent system comprising: an elongate...,1. A ridge vent system comprising: an elongate...,10281189;10739055;10845115;9695594
8925263,8925263,2015,1,NaN,H02S20/23;F16B5/0607;F16B2/065;F24S25/632;H02S...,F16B2005/0678;Y02B10/20;F24S2025/6008;F16B2/12...,E04D13/18,E04H14/00;H01L31/042;F24J2/52,0,0,...,5,2,9,9,7,6,1,"1. A photovoltaic module mounting assembly, co...","1. A photovoltaic module mounting assembly, co...",10014818;10041254;10097132;10128790;10205418;1...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
RE46435,RE46435,2017,6,SANDISK TECHNOLOGIES LLC,G11C5/025;G11C8/08;H01L27/1021;H01L27/2409;H01...,G11C16/0483;H01L27/112;H01L45/085;H01L45/04;H0...,G11C5/02,G11C8/08,0,0,...,1,1,1,4,2,1,1,"1. A nonvolatile memory device, comprising.Iad...","1. A nonvolatile memory device, comprising.Iad...",10061633;10283493;10381322;10453798;10510738;1...
RE46441,RE46441,2017,6,NOVARTIS AG,B01F33/811;A61K9/1075;B01F25/44;B01F23/41;A61K...,A61K2039/55566,A61K31/01,A61K9/107;B01F5/06;B01F13/10;B01F3/08;A61K39/3...,0,0,...,1,1,1,1,1,1,1,1. A method for the manufacture of a squalene-...,1. A method for the manufacture of a squalene-...,10159637;10463615;10813872
RE46456,RE46456,2017,6,"RICOH COMPANY, LTD.",G06F1/325;G06F1/3203,NaN,G06F13/24,G06F1/32,0,0,...,1,1,1,1,1,1,1,1. An image processing device comprising: a fi...,1. An image processing device comprising: a fi...,10631051
RE46465,RE46465,2017,7,KAWASAKI JUKOGYO KABUSHIKI KAISHA,H01L21/67766,Y10S414/139;Y10S414/135;Y10S414/137,H01L21/677,NaN,0,0,...,1,1,1,1,2,1,1,.[.1. A wafer transfer apparatus for transferr...,.[.1. A wafer transfer apparatus for transferr...,RE47145;RE47909;RE48031


In [217]:
temp.rename(columns={"year": "granted_year", "main ipc": "main_ipc", "sub ipc": "sub_ipc"}).to_csv(os.path.join(data_dir, "collection_" + "".join([config_keywords, config_ipcs, config_period]) + ".csv"))

In [140]:
args = argparse.Namespace(
    data_type="class+claim",
#     data_file = "collection_[H01L,H10][2017].csv",
    target_ipc=["H01L", "H10"],
    pred_type="classification",
    n_TC = 5,
    use_pretrained_tokenizer=False,
    do_train=None,
    do_tune=None,
    n_folds=None,
    ipc_level=4,
    batch_size=16,
    max_epochs=50,
    use_accelerator=None,
    do_save=True,
    n_gpus=4,
    light=True,
#     config_file=os.path.join(root_dir, "configs", "USED_configs", "[CONFIGS]2023-04-11_00:39.json"),
    config_file=None,
    eval_train_set=False)

data_dir = os.path.join(master_dir, "data")
model_dir = os.path.join(root_dir, "models")
result_dir = os.path.join(root_dir, "results")
config_dir = os.path.join(root_dir, "configs")

# args = parser.parse_args()
if args.config_file is not None:
    config_file = args.config_file
else:
    config_file = os.path.join(config_dir, "configs_light.json") if args.light else os.path.join(config_dir, "configs.json")
configs = DotDict().load(config_file)
org_config_keys = {key: list(configs[key].keys()) for key in configs.keys()}

instant_configs = {key: value for (key, value) in vars(args).items() if value is not None} # if any argument passed when main.py executed
instant_configs_for_update = {configkey: {key: value for (key,value) in instant_configs.items() if key in org_config_keys[configkey]} for configkey in org_config_keys.keys()}
for key, value in configs.items():
    value.update(instant_configs_for_update[key])

# regex_ipc = re.compile('[A-Z](?![\\D])')
# if regex_ipc.match(configs.data.target_ipc) is None:
#     configs.data.update({"target_ipc": "ALL"})
# elif len(configs.data.target_ipc) > 5:
#     configs.data.update({"target_ipc": configs.data.target_ipc[:4]})

if configs.model.model_type == "enc-pred-dec":
    configs.train.loss_weights["recon"] = configs.train.loss_weights["recon"] / sum(configs.train.loss_weights.values())
    configs.train.loss_weights["y"] = 1 - configs.train.loss_weights["recon"]
elif configs.model.model_type == "enc-pred":
    configs.train.loss_weights = {"recon": 0, "y": 1}
elif configs.model.model_type == "enc-dec":
    configs.train.loss_weights = {"recon": 1, "y": 0}

if configs.train.use_accelerator:
    accelerator = Accelerator()
    device_ids = list(range(torch.cuda.device_count()))
    device = accelerator.device
    configs.train.update({"accelerator": accelerator})
else:
    if torch.cuda.is_available():
        device_ids = list(range(torch.cuda.device_count()))
        gpu_usages = [np.sum([float(usage.split("uses")[-1].replace(" ","").replace("MB","")) for usage in torch.cuda.list_gpu_processes(id).split("GPU memory") if not usage=="" and "no processes are running" not in usage]) for id in device_ids]
        device_ids = np.argsort(gpu_usages)[:configs.train.n_gpus]
        device_ids = list(map(lambda x: torch.device('cuda', x),list(device_ids)))
        device = device_ids[0] # main device
        torch.cuda.set_device(device)
    else:
        device = torch.device('cpu')
        device_ids = []

configs.data.update({"root_dir": root_dir,
                        "data_dir": data_dir,
                        "model_dir": model_dir,
                        "result_dir": result_dir,
                        "pretrained_enc": configs.model.pretrained_enc,
                        "pretrained_dec": configs.model.pretrained_dec,
                        "data_nrows": None})
configs.train.update({"device": device,
                        "device_ids": device_ids,
                        "root_dir": root_dir,
                        "data_dir": data_dir,
                        "model_dir": model_dir,
                        "use_keywords": configs.data.use_keywords,
                        "early_stop_patience": int(0.3*configs.train.max_epochs)})
configs.model.update({"device": device,
                        "device_ids": device_ids,
                        "n_directions": 2 if configs.model.bidirec else 1,
                        "use_accelerator": configs.train.use_accelerator})

## Set hyperparameters for model training (To be TUNED)
if configs.train.do_train and configs.train.do_tune:
    n_layers = configs.model.n_layers = None
    d_embedding = configs.model.d_embedding = None
    d_enc_hidden = configs.model.d_enc_hidden = None
    d_pred_hidden = configs.model.d_pred_hidden = None
    learning_rate = configs.train.learning_rate = None
    batch_size = configs.train.batch_size = None
    config_name = "HPARAM_TUNING"
    final_model_path = None
else:
    n_layers = configs.model.n_layers
    d_embedding = configs.model.d_embedding
    d_enc_hidden = configs.model.d_enc_hidden
    d_pred_hidden = configs.model.d_pred_hidden
    d_latent = configs.model.d_latent

    key_components = {"data": ["target_ipc", "vocab_size"], "model": ["n_layers", "d_enc_hidden", "d_pred_hidden", "d_latent", "d_embedding", "d_ff", "n_head", "d_head"], "train": ["learning_rate", "batch_size", "max_epochs"]}
    config_name = ""
    for key in key_components.keys():
        for component in key_components[key]:
            config_name += "["+str(configs[key][component])+component+"]"
    final_model_path = os.path.join(model_dir, f"[Final_model]{config_name}.ckpt")

configs.train.update({"config_name": config_name,
                        "final_model_path": final_model_path})

In [161]:
configs = DotDict().load(config_file)

### 파일명 실험

In [156]:
target_period = [2015, 2017] # format: [year_after, year_before] or [-1, year_before] or [year_after, -1]; "-1" indicates the oldest or latest year
target_ipcs = ["H01L", "H10"]
# target_keywords = ["semiconductor", "silicon", "chip"]
target_keywords = ""

In [210]:
dataset_path

'/home2/glee/dissertation/1_tech_gen_impact/master/Tech_Gen/data/pickled_dataset/[DATASET][semiconductor,silicon,chip][H01L,H10][2015-2017]data[3]ipc_level[30]max_seq_len_class[200]max_seq_len_claim[1500]vocab_size.pickle'

In [162]:
configs.data

DotDict({'data_type': 'class', 'pred_type': 'classification', 'target_period': [2015, 2017], 'target_ipc': ['H01L', 'H10'], 'target_keywords': ['semiconductor', 'silicon', 'chip'], 'pred_target': 'citation', 'ipc_level': 3, 'claim_level': 1, 'class_level': 3, 'n_TC': 5, 'use_keywords': False, 'max_seq_len_class': 30, 'max_seq_len_claim': 200, 'vocab_size': 1500, 'use_pretrained_tokenizer': False})

In [165]:
config_period = "["+"-".join([str(year) for year in configs.data.target_period])+"]"
config_ipcs = str(configs.data.target_ipc).replace("\'","").replace(" ","")
config_keywords = str(configs.data.target_keywords).replace("\'","").replace(" ","")
dataset_config_name = "".join([config_keywords, config_ipcs, config_period])

[semiconductor,silicon,chip][H01L,H10][2015-2017]


In [206]:
dataset_config_name = "".join([config_keywords, config_ipcs, config_period]) + "data"
config_period = "["+"-".join([str(year) for year in configs.data.target_period])+"]"
config_ipcs = str(configs.data.target_ipc).replace("\'","").replace(" ","")
config_keywords = str(configs.data.target_keywords).replace("\'","").replace(" ","")
for component in dataset_key_components:
    dataset_config_name += f"[{str(configs.data[component])}]{component}"

In [207]:
dataset_config_name

'[semiconductor,silicon,chip][H01L,H10][2015-2017]data[3]ipc_level[30]max_seq_len_class[200]max_seq_len_claim[1500]vocab_size'

In [208]:
dataset_path = os.path.join(data_dir, "pickled_dataset", "[DATASET]"+dataset_config_name+".pickle")

In [209]:
dataset_path

'/home2/glee/dissertation/1_tech_gen_impact/master/Tech_Gen/data/pickled_dataset/[DATASET][semiconductor,silicon,chip][H01L,H10][2015-2017]data[3]ipc_level[30]max_seq_len_class[200]max_seq_len_claim[1500]vocab_size.pickle'

In [141]:
final_model_path

'/home2/glee/dissertation/1_tech_gen_impact/class2class/Tech_Gen/models/[Final_model][H01Ltarget_ipc][1500vocab_size][4n_layers][16d_enc_hidden][8d_pred_hidden][64d_latent][128d_embedding][16d_ff][4n_head][32d_head][0.0005learning_rate][16batch_size]ax_epochs].ckpt'

In [168]:
dataset_path

'/home2/glee/dissertation/1_tech_gen_impact/master/Tech_Gen/data/pickled_dataset/[tech_dataset]data_type=class+claim-pred_type=classification-target_ipc=H01L-target_type=citation-ipc_level=4-claim_level=1-class_level=3-n_TC=5-use_keywords=False-max_seq_len_class=30-max_seq_len_claim=200-vocab_size=1500.pickle'

In [185]:
dataset_key_components = ["ipc_level", "max_seq_len_class", "max_seq_len_claim", "vocab_size"]

In [180]:
org_config_keys["data"]

['data_type',
 'data_file',
 'pred_type',
 'target_ipc',
 'target_type',
 'ipc_level',
 'claim_level',
 'class_level',
 'n_TC',
 'use_keywords',
 'max_seq_len_class',
 'max_seq_len_claim',
 'vocab_size',
 'use_pretrained_tokenizer']

In [167]:
org_config_keys_temp

['data_type',
 'pred_type',
 'target_ipc',
 'target_type',
 'ipc_level',
 'claim_level',
 'class_level',
 'n_TC',
 'use_keywords',
 'max_seq_len_class',
 'max_seq_len_claim',
 'vocab_size']

In [213]:
temp = "collection_" + "".join([config_keywords, config_ipcs, config_period]) + ".csv"

'collection_[semiconductor,silicon,chip][H01L,H10][2015-2017].csv'

In [ ]:
dataset_key_components = 

# PART 2: Dataset setting

In [3]:
tstart = time.time()
org_config_keys_temp = copy.copy(org_config_keys["data"])
org_config_keys_temp.pop(org_config_keys_temp.index("use_pretrained_tokenizer"))
org_config_keys_temp.pop(org_config_keys_temp.index("data_file"))
dataset_config_name = "-".join([str(key)+"="+str(value) for (key,value) in configs.data.items() if key in org_config_keys_temp])
dataset_path = os.path.join(data_dir, "pickled_dataset", "[tech_dataset]"+dataset_config_name+".pickle")
if os.path.exists(dataset_path) and args.do_save is False:
    print("Load pickled dataset...")
    with open(dataset_path, "rb") as f:
        tech_dataset = pickle.load(f)   # Load pickled dataset if dataset with same configuration already saved
        if tech_dataset.pretrained_enc != configs.data.pretrained_enc or tech_dataset.pretrained_dec != configs.data.pretrained_dec:
                tech_dataset.pretrained_enc = configs.data.pretrained_enc
                tech_dataset.pretrained_dec = configs.data.pretrained_dec
                tech_dataset.tokenizers = tech_dataset.get_tokenizers()
        for tk in tech_dataset.tokenizers.values():
            if "vocab_size" not in dir(tk):
                tk.vocab_size = tk.get_vocab_size()
    print("Pickled dataset loaded")
else:
    print("Make dataset...")
    tech_dataset = TechDataset(configs.data)
#     with open(dataset_path, "wb") as f:
#         tech_dataset.rawdata = None
#         pickle.dump(tech_dataset, f)
tend = time.time()
print(f"{np.round(tend-tstart,4)} sec elapsed for loading patents for class [{configs.data.target_ipc}]")

Make dataset...



Tokenizer is trained and saved
62.5476 sec elapsed for loading patents for class [H01L]


In [4]:
tech_dataset.data

,number,main_ipc,sub_ipc,ipcs,claims,TC5,TC5_digitized,class
number,,,,,,,,
9853235,9853235,H01L51/52,"[H01L27/32, H05B33/04]","[H01L51/52, H01L27/32, H05B33/04]",1. A display device comprising: a light emitti...,0,0,106
9854199,9854199,H04N05/76,"[G11B27/00, G11B27/024, G11B27/032, G11B27/034...","[H04N05/76, G11B27/00, G11B27/024, G11B27/032,...","1. A method for a digital video recorder, comp...",0,0,109
9851599,9851599,G02F01/1335,"[G02B05/20, G02F01/1343, G09G03/34, G09G03/36,...","[G02F01/1335, G02B05/20, G02F01/1343, G09G03/3...",1. A color display device for displaying an n-...,0,0,95
9851864,9851864,G06F17/21,"[G06F03/041, G06F03/0481, G06F03/0485, G06F03/...","[G06F17/21, G06F03/041, G06F03/0481, G06F03/04...",1. A method comprising: identifying content to...,0,0,99
9852488,9852488,A63F09/24,"[A63F13/00, G06F17/00, G06F19/00, G06Q50/34, G...","[A63F09/24, A63F13/00, G06F17/00, G06F19/00, G...",1. A computer implemented method of managing b...,8,1,13
...,...,...,...,...,...,...,...,...
9537605,9537605,H04K03/00,[H04B01/04],"[H04K03/00, H04B01/04]","1. An ultra-wideband, high-power, solid-state ...",1,0,109
9538636,9538636,H05K01/02,"[H05K01/03, H05K01/14, H05K01/18, H05K03/00, H...","[H05K01/02, H05K01/03, H05K01/14, H05K01/18, H...",1. An apparatus comprising: a substrate compri...,2,0,110
9536977,9536977,H01L29/66,"[H01L21/332, H01L21/336, H01L21/8238, H01L29/739]","[H01L29/66, H01L21/332, H01L21/336, H01L21/823...",1. A semiconductor device comprising: a precur...,5,1,106


In [ ]:
configs.model.update({"tokenizers": tech_dataset.tokenizers,
                    "n_enc_seq_claim": tech_dataset.max_seq_len_claim,
                    "n_dec_seq_claim": tech_dataset.max_seq_len_claim,
                    "n_enc_seq_class": tech_dataset.max_seq_len_class,
                    "n_dec_seq_class": tech_dataset.max_seq_len_class,
                    "n_outputs": 1 if configs.data.pred_type=="regression" else tech_dataset.n_outputs,
                    "i_padding": tech_dataset.tokenizers["class_enc"].token_to_id("<PAD>")})

# PART 3: Training

In [ ]:
sampler = CVSampler(tech_dataset, n_folds=configs.train.n_folds, test_ratio=0.1, stratify=True)
cv_idx = sampler.get_idx_dict()
print(f"#Samples\nTrain: {len(cv_idx[0]['train'])}, Validation: {len(cv_idx[0]['val'])}, Test: {len(cv_idx[0]['test'])}")

## PART 3-2: Dataset construction and model training

In [ ]:
## Construct datasets
train_idx = cv_idx[0]['train']
val_idx = cv_idx[0]['val']
test_idx = cv_idx[0]['test']
whole_idx = np.concatenate([train_idx, val_idx])

train_dataset = Subset(tech_dataset, train_idx)
val_dataset = Subset(tech_dataset, val_idx)
test_dataset = Subset(tech_dataset, test_idx)
whole_dataset = Subset(tech_dataset, whole_idx)

train_loader = DataLoader(train_dataset, batch_size=configs.train.batch_size, shuffle=True, num_workers=4, drop_last=True)
val_loader = DataLoader(val_dataset, batch_size=configs.train.batch_size if len(val_idx)>configs.train.batch_size else len(val_idx), shuffle=True, num_workers=4, drop_last=False)
test_loader = DataLoader(test_dataset, batch_size=configs.train.batch_size if len(test_idx)>configs.train.batch_size else len(test_idx), shuffle=False, num_workers=4)
whole_loader = DataLoader(whole_dataset, batch_size=configs.train.batch_size, shuffle=False, num_workers=4)

## Load best model or build model
final_model = build_model(configs.model, tokenizers=tech_dataset.tokenizers)

### Early stopping for multi criteria 실험

In [133]:
class EarlyStopping:
    def __init__(self, patience=10, verbose=True, delta=0, path='../models/checkpoint.ckpt'):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path

    def __call__(self, val_loss, model):
        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
                # Load latest saved model
                model = self.load_model(model)
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

        return model

    def save_checkpoint(self, val_loss, model):
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...\n')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

    def load_model(self, model):
        saved_model = copy.copy(model)
        saved_model.load_state_dict(torch.load(self.path))
        return saved_model

class EarlyStopping_multi(EarlyStopping):
    def __init__(self, patience=10, verbose=True, delta=0, path='../models/checkpoint.ckpt', criteria=None):
        super().__init__(patience=10)
        self.best_scores = None
        self.delta = 0.005
        self.deltas = None
        if criteria is not None:
            self.criteria = criteria
        else:
            self.criteria = ["total", "recon", "y"]
        self.val_losses_min = {criterion: np.Inf for criterion in self.criteria}

    def __call__(self, model, val_losses={}):
        scores = {k: -val_losses[k] for k in self.criteria}
        if self.best_scores is not None:
            print([(self.best_scores[k], self.deltas[k], self.best_scores[k]+self.deltas[k], scores[k]) for k in self.criteria])

        if self.best_scores is None:
            self.best_scores = scores
            self.deltas = {k: self.best_scores[k]*self.delta for k in self.criteria}
            self.save_checkpoint(val_losses, model)
        elif any([scores[k] < self.best_scores[k] + self.deltas[k] for k in self.criteria]):
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
                # Load latest saved model
                model = self.load_model(model)
        else:
            self.best_scores = scores
            self.deltas = {k: self.best_scores[k]*self.delta for k in self.criteria}
            self.save_checkpoint(val_losses, model)
            self.counter = 0

        return model

    def save_checkpoint(self, val_losses, model):
        if self.verbose:
            for criterion in self.criteria:
                print(f'Validation loss[{criterion}] decreased ({self.val_losses_min[criterion]:.6f} --> {val_losses[criterion]:.6f})')
            print("\n")
        torch.save(model.state_dict(), self.path)
        self.val_losses_min = val_losses

In [134]:
es = EarlyStopping(patience=11, verbose=True, path="../models/es_test.ckpt")
es_multi = EarlyStopping_multi(patience=10, criteria=["total", "kld"])

In [123]:
losses_ = {"total": 10, "recon": 8, "y": 2, "kld": 100}

In [103]:
losses_ = {"total": 10, "recon": 7, "y": 1}

In [124]:
model = es_multi(model, val_losses=losses_)

Validation loss[total] decreased (inf --> 10.000000)
Validation loss[kld] decreased (inf --> 100.000000)




In [125]:
es_multi.best_scores

{'total': -10, 'kld': -100}

In [126]:
es_multi.deltas

{'total': -0.05, 'kld': -0.5}

In [128]:
es_multi.val_losses_min

{'total': 10, 'recon': 8, 'y': 2, 'kld': 100}

In [138]:
0.024216 * 1.001

0.024240216

In [136]:
0.024282

0.024282

### Variational 실험

In [25]:
import importlib
import models
importlib.reload(models)
from models import CLS2CLS, VCLS2CLS

In [26]:
model = VCLS2CLS(configs.model, tokenizers=tech_dataset.tokenizers).to(configs.model.device)

In [ ]:
batch_data = to_device(next(iter(train_loader)), device=configs.model.device)

In [ ]:
enc_outputs, z, mu, logvar = model.encode(batch_data["text_inputs"])

In [ ]:
pred_outputs = model.predict(z)

In [ ]:
dec_outputs = model.decode(z, enc_outputs["class"], batch_data["text_outputs"], teach_force_ratio=0.5)

In [ ]:
dec_outputs.shape

### Variational 빼고 Attention 추가해서 실험

In [ ]:
import importlib
import models
importlib.reload(models)
from models import CLS2CLS, VCLS2CLS

In [ ]:
model = CLS2CLS(configs.model, tokenizers=tech_dataset.tokenizers).to(configs.model.device)

In [ ]:
batch_data = to_device(next(iter(train_loader)), device=configs.model.device)

In [ ]:
enc_outputs, z, mu, logvar = model.encode(batch_data["text_inputs"])

In [ ]:
enc_outputs={}
enc_outputs["claim"], *_ = model.encoders["claim"](**batch_data["text_inputs"]["claim"])
enc_outputs["class"], hidden = model.encoders["class"](batch_data["text_inputs"]["class"])

In [ ]:
enc_outputs["class"].shape

In [ ]:
hidden.shape

In [ ]:
hidden.permute(1,0,2).contiguous().view(16, -1)

In [ ]:
model.d_hidden * model.n_directions * model.n_layers

In [ ]:
model.d_hidden

In [ ]:
model.pool(enc_outputs["claim"]).shape 

In [ ]:
enc_outputs["class"].shape

In [ ]:
model.mu

In [ ]:
pred_outputs = model.predict(z)

In [ ]:
dec_outputs = model.decode(z, enc_outputs["class"], batch_data["text_outputs"], teach_force_ratio=0.5)

In [ ]:
def kl_anneal_function(step, func_type="logistic", k=0.0025, x0=2500):
    if func_type == 'logistic':
        return float(1/(1+np.exp(-k*(step-x0))))
    elif func_type == 'linear':
        return min(1, step/x0)

In [ ]:
kl_anneal_function(0)

In [ ]:
np.max(tech_dataset.X_class.apply(lambda x: len(x)))

In [ ]:
whole_loader = DataLoader(whole_dataset, batch_size=len(whole_dataset))

In [ ]:
wholebatch = next(iter(whole_loader))

### predictor parameter update 실험, z-sampling 실험

In [ ]:
class_weights = torch.tensor(np.unique(tech_dataset.Y[whole_idx], return_counts=True)[1][::-1].copy()).to(device)
class_weights = class_weights / class_weights.sum()
from utils import KLDLoss
from parallel import DataParallel
model_params = configs.model
loss_recon = torch.nn.CrossEntropyLoss(ignore_index=model_params['i_padding'])
loss_y = torch.nn.MSELoss() if model_params['n_outputs']==1 else torch.nn.NLLLoss(weight=class_weights)
loss_kld = KLDLoss()
optimizer = torch.optim.AdamW(filter(lambda p: p.requires_grad, final_model.module.parameters()), lr=configs.train['learning_rate'])

In [ ]:
for p in final_model.module.predictor.parameters():
    print(p.requires_grad)

In [ ]:
batch_data = next(iter(train_loader))
batch_data = to_device(batch_data, device)

In [ ]:
outputs = final_model.module(batch_data["text_inputs"], batch_data["text_outputs"])
preds_recon = outputs["dec_outputs"].permute(0,2,1)
trues_recon = batch_data["text_outputs"]
preds_y = outputs["pred_outputs"]
trues_y = batch_data["targets"]
preds_mu = outputs["mu"]
preds_logvar = outputs["logvar"]

In [ ]:
loss = loss_recon(preds_recon, trues_recon) + loss_kld(preds_mu, preds_logvar) + loss_y(preds_y, trues_y)
print(loss)

In [ ]:
enc_outputs, z, mu, logvar = final_model.module.encode(batch_data["text_inputs"])
pred_outputs = final_model.module.predictor(z)

In [ ]:
z[:10,:10]

In [ ]:
z.sum()

In [ ]:
for p in final_model.module.predictor.parameters():
    print(p.sum())
for p in final_model.module.predictor.parameters():
    print(p)

In [ ]:
loss.backward()
optimizer.step()

In [ ]:
outputs = final_model.module(batch_data["text_inputs"], batch_data["text_outputs"])
preds_recon = outputs["dec_outputs"].permute(0,2,1)
trues_recon = batch_data["text_outputs"]
preds_y = outputs["pred_outputs"]
trues_y = batch_data["targets"]
preds_mu = outputs["mu"]
preds_logvar = outputs["logvar"]

In [ ]:
loss = loss_recon(preds_recon, trues_recon) + loss_kld(preds_mu, preds_logvar) + loss_y(preds_y, trues_y)
print(loss)

In [ ]:
enc_outputs, z, mu, logvar = final_model.module.encode(batch_data["text_inputs"])
pred_outputs = final_model.module.predictor(z)

In [ ]:
z[:10,:10]

In [ ]:
z.sum()

In [ ]:
for p in final_model.module.predictor.parameters():
    print(p.sum())
for p in final_model.module.predictor.parameters():
    print(p)

In [ ]:
final_model.module.freeze(module_name="predictor")
for p in final_model.module.predictor.parameters():
    print(p.requires_grad)

In [ ]:
optimizer = torch.optim.AdamW(filter(lambda p: p.requires_grad, final_model.module.parameters()), lr=configs.train['learning_rate'])

In [ ]:
loss.backward()
optimizer.step()

In [ ]:
outputs = final_model.module(batch_data["text_inputs"], batch_data["text_outputs"])
preds_recon = outputs["dec_outputs"].permute(0,2,1)
trues_recon = batch_data["text_outputs"]
preds_y = outputs["pred_outputs"]
trues_y = batch_data["targets"]
preds_mu = outputs["mu"]
preds_logvar = outputs["logvar"]

In [ ]:
loss = loss_recon(preds_recon, trues_recon) + loss_kld(preds_mu, preds_logvar) + loss_y(preds_y, trues_y)
print(loss)

In [ ]:
enc_outputs, z, mu, logvar = final_model.module.encode(batch_data["text_inputs"])
pred_outputs = final_model.module.predictor(z)

In [ ]:
z[:10,:10]

In [ ]:
z.sum()

In [ ]:
for p in final_model.module.predictor.parameters():
    print(p.sum(), p.requires_grad)
for p in final_model.module.predictor.parameters():
    print(p)

In [ ]:
for p in final_model.module.predictor.parameters():
    print(p.sum(), p.requires_grad)
for p in final_model.module.predictor.parameters():
    print(p)

In [ ]:
final_model.module.freeze(module_name="predictor", defreeze=True)
for p in final_model.module.predictor.parameters():
    print(p.requires_grad)

In [ ]:
optimizer = torch.optim.AdamW(filter(lambda p: p.requires_grad, final_model.module.parameters()), lr=configs.train['learning_rate'])

In [ ]:
loss.backward()
optimizer.step()

In [ ]:
outputs = final_model.module(batch_data["text_inputs"], batch_data["text_outputs"])
preds_recon = outputs["dec_outputs"].permute(0,2,1)
trues_recon = batch_data["text_outputs"]
preds_y = outputs["pred_outputs"]
trues_y = batch_data["targets"]
preds_mu = outputs["mu"]
preds_logvar = outputs["logvar"]

In [ ]:
loss = loss_recon(preds_recon, trues_recon) + loss_kld(preds_mu, preds_logvar) + loss_y(preds_y, trues_y)
print(loss)

In [ ]:
enc_outputs, z, mu, logvar = final_model.module.encode(batch_data["text_inputs"])
pred_outputs = final_model.module.predictor(z)

In [ ]:
z[:10,:10]

In [ ]:
z.sum()

In [ ]:
for p in final_model.module.predictor.parameters():
    print(p.sum(), p.requires_grad)
for p in final_model.module.predictor.parameters():
    print(p)

## PART 3-3: Training evaluation

In [ ]:
if args.eval_train_set:
    ## Evaluation on train dataset
    print("Validate model on train dataset")
    # trues_recon_train, preds_recon_train, trues_y_train, preds_y_train = validate_model(final_model, whole_loader, configs.model, configs.train)
    val_res_train = validate_model_mp(final_model, whole_dataset, mp=mp, model_params=configs.model, train_params=configs.train)
    trues_recon_train = np.concatenate([res["recon"]["true"] for res in val_res_train.values()])
    preds_recon_train = np.concatenate([res["recon"]["pred"] for res in val_res_train.values()])
    trues_y_train = np.concatenate([res["y"]["true"] for res in val_res_train.values()])
    preds_y_train = np.concatenate([res["y"]["pred"] for res in val_res_train.values()])

    eval_recon_train = perf_eval("TRAIN_SET", trues_recon_train, preds_recon_train, configs=configs, pred_type='generative', tokenizer=final_model.module.tokenizer)
    eval_recon_train = perf_eval("TRAIN_SET", trues_recon_train, preds_recon_train, configs=configs, pred_type='generative', tokenizer=final_model.module.tokenizer)
    eval_y_train = perf_eval("TRAIN_SET", trues_y_train, preds_y_train, configs=configs, pred_type=configs.data.pred_type)
    if configs.data.pred_type == "classification":
        eval_y_train, confmat_y_train = eval_y_train
else:
    eval_recon_train = eval_y_train = None

## Evaluation on test dataset
print("Validate model on test dataset")
# trues_recon_test, preds_recon_test, trues_y_test, preds_y_test = validate_model(final_model, test_loader, configs.model, configs.train)
val_res_test = validate_model_mp(final_model, test_dataset, mp=mp, batch_size=64, model_params=configs.model, train_params=configs.train)
trues_recon_test = np.concatenate([res["recon"]["true"] for res in val_res_test.values()])
preds_recon_test = np.concatenate([res["recon"]["pred"] for res in val_res_test.values()])
trues_y_test = np.concatenate([res["y"]["true"] for res in val_res_test.values()])
preds_y_test = np.concatenate([res["y"]["pred"] for res in val_res_test.values()])

eval_recon_test = perf_eval("TEST_SET", trues_recon_test, preds_recon_test, configs=configs,  pred_type='generative', tokenizer=final_model.module.tokenizer)
eval_y_test = perf_eval("TEST_SET", trues_y_test, preds_y_test, configs=configs, pred_type=configs.data.pred_type)
if configs.data.pred_type == "classification":
    eval_y_test, confmat_y_test = eval_y_test

eval_recon_res = pd.concat([eval_recon_train, eval_recon_test], axis=0)
eval_y_res = pd.concat([eval_y_train, eval_y_test], axis=0)
if configs.data.pred_type == "classification":
    confmat_y_res = pd.concat([confmat_y_train, confmat_y_test], axis=0)
